ใน ep นี้ เราจะ

# 0. Install

Install Library ที่จำเป็น

In [39]:
## Colab
! curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import

Import Library ที่จะใช้ ในที่นี้คือ fastai.text และ sklearn_text ที่ใช้ในงาน [NLP](https://www.bualabs.com/archives/119/what-is-nlp-natural-language-processing-nlp-task-in-thai-nlp-ep-1/)

In [0]:
from fastai import *
from fastai.text import *
import sklearn.feature_extraction.text as sklearn_text
from matplotlib import pyplot as plt
import seaborn as sns

# 2. Dataset

ในเคสนี้เราจะใช้ IMDB Movie Review เป็นรีวิวหนังจากเว็บ IMDB ที่มีข้อความ และ คะแนนว่าเป็นแง่บวก หรือแง่ลบ

ในการพัฒนา เราจะใช้ Dataset ชุดเล็กก่อน จะได้เร็ว เมื่อเทสทุกอย่างเรียบร้อย แล้วจึงขยับไปใช้ Dataset ชุดเต็ม 

In [42]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/content/data/imdb_sample')

In [43]:
path.ls()

[PosixPath('/content/data/imdb_sample/texts.csv')]

In [44]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [45]:
movie_reviews = (TextList.from_csv(path, "texts.csv", cols="text")
                        .split_from_df(col=2)
                        .label_from_df(cols=0))

len(movie_reviews.train.x), len(movie_reviews.valid.x)

(800, 200)

In [46]:
v = movie_reviews.vocab.itos
vocab_len = len(v)
vocab_len

6008

# 3. Preprocessing

## 3.1 Create Training N-Gram Matrix


ประกาศตัวแปร ที่จำเป็น

In [0]:
min_n = 1
max_n = 3

j_indices = []
indptr = []
values = []
indptr.append(0)
num_tokens = vocab_len

itongram = dict()
ngramtoi = dict()

In [0]:
for i, doc in enumerate(movie_reviews.train.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k+n]
            
            if str(ngram) not in ngramtoi: 
                if len(ngram) == 1: 
                    num = ngram[0]
                    ngramtoi[str(ngram)] = num
                    itongram[num] = ngram
                else:
                    ngramtoi[str(ngram)] = num_tokens
                    itongram[num_tokens] = ngram
                    num_tokens += 1
            this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))


In [49]:
len(values), len(j_indices), len(indptr) - 1, len(ngramtoi)

(678885, 678885, 800, 260374)

In [0]:
train_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr), 
                                                 shape=(len(indptr) - 1, len(ngramtoi)), 
                                                 dtype=int)

In [51]:
train_ngram_doc_matrix

<800x260374 sparse matrix of type '<class 'numpy.int64'>'
	with 678885 stored elements in Compressed Sparse Row format>

In [52]:
len(ngramtoi), len(itongram)

(260374, 260374)

ลองดูใน N-Gram Term-Document Matrix ที่ได้

### ตัวอย่าง 1

In [53]:
itongram[20005]

array([ 15,   9, 710])

In [54]:
ngramtoi[str(np.array([15, 9, 710]))]

20005

In [55]:
v[15], v[9], v[710]

('to', 'the', 'leads')

### ตัวอย่าง 2

In [62]:
itongram[20050]

array([  14,    5, 2808])

In [63]:
ngramtoi[str(np.array([14, 5, 2808]))]

20050

In [64]:
v[14], v[5], v[2808]

('of', 'xxmaj', 'celeste')

### ตัวอย่าง 3

In [65]:
itongram[20080]

array([ 23, 796,  11])

In [66]:
ngramtoi[str(np.array([23, 796,  11]))]

20080

In [67]:
v[23], v[796], v[11]

("'s", 'career', ',')

## 3.2 Create Valid N-Gram Matrix

In [0]:
j_indices = []
indptr = []
values = []
indptr.append(0)

for i, doc in enumerate(movie_reviews.valid.x):
    feature_counter = Counter(doc.data)
    j_indices.extend(feature_counter.keys())
    values.extend(feature_counter.values())
    this_doc_ngrams = list()

    m = 0
    for n in range(min_n, max_n + 1):
        for k in range(vocab_len - n + 1):
            ngram = doc.data[k: k + n]
            if str(ngram) in ngramtoi:
                this_doc_ngrams.append(ngramtoi[str(ngram)])
            m += 1

    ngram_counter = Counter(this_doc_ngrams)
    j_indices.extend(ngram_counter.keys())
    values.extend(ngram_counter.values())
    indptr.append(len(j_indices))

In [0]:
valid_ngram_doc_matrix = scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, len(ngramtoi)),
                                   dtype=int)

In [70]:
valid_ngram_doc_matrix

<200x260374 sparse matrix of type '<class 'numpy.int64'>'
	with 121600 stored elements in Compressed Sparse Row format>

In [71]:
train_ngram_doc_matrix

<800x260374 sparse matrix of type '<class 'numpy.int64'>'
	with 678885 stored elements in Compressed Sparse Row format>

## 3.3 Save Data

In [0]:
scipy.sparse.save_npz("train_ngram_doc_matrix.npz", train_ngram_doc_matrix)

In [0]:
scipy.sparse.save_npz("valid_ngram_doc_matrix.npz", valid_ngram_doc_matrix)

In [0]:
with open('itongram.pickle', 'wb') as handle:
    pickle.dump(itongram, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('ngramtoi.pickle', 'wb') as handle: 
    pickle.dump(ngramtoi, handle, protocol=pickle.HIGHEST_PROTOCOL)

## 3.4 Load Data

In [0]:
train_ngram_doc_matrix = scipy.sparse.load_npz("train_ngram_doc_matrix.npz")
valid_ngram_doc_matrix = scipy.sparse.load_npz("valid_ngram_doc_matrix.npz")

In [0]:
with open("itongram.pickle", "rb") as handle:
    b = pickle.load(handle)
with open("ngramtoi.pickle", "rb") as handle:
    b = pickle.load(handle)

# สรุป

* เราได้เรียนรู้

# Credit

* https://www.youtube.com/watch?v=dt7sArnLo1g&list=PLtmWHNX-gukKocXQOkQjuVxglSDYWsSh9&index=6&t=0s
* https://www.bualabs.com/archives/3000/sentiment-classification-imdb-movie-reviews-with-naive-bayes-logistic-regression-nlp-ep-5/
* https://www.bualabs.com/archives/926/sentiment-analysis-imdb-movie-review-ulmfit-sentiment-analysis-ep-1/
* https://docs.fast.ai/text.data.html
* https://docs.python.org/2/library/collections.html
* https://en.wikipedia.org/wiki/Naive_Bayes_classifier
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
* https://en.wikipedia.org/wiki/N-gram
* https://web.stanford.edu/~jurafsky/slp3/3.pdf